### Desafio 1
  - Lauro, Rodrigo (ing.rodrigo.lauro@gmail.com)
  - Nº SIU a2120

In [ ]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[210])

To what follows, our moderator has already answered the charge of 
arrogance more ably that I could have done so, so I will confine
myself to answering the charge of illogic.
 


This is how everyone in the western intellectual tradition is, or was,
taught to think. It is the fundamental premis "A is not not-A". If a thing
is true then its converse is necessarilly false. Without this basic 
asumption theology and science as we know them are alike impossible. We
should distinguish the strong and weak meanings of the word "believe",
however. The weak sense means I am not sure. "I believe Tom went to 
the library." (but he could have gone to the track). The strong sense
means I am so certain that I use it as a basis of thought. "I believe 
that nature operates according to certain fundamental laws." (despite 
the fact that nature *appears* capricious and unpredictable). Christian
belief is of the strong kind. (Though Christians may well hold beliefs
of the weak kind on any number of theol

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


### Punto 1

#### Observaciones:

  * Se observa que en los documentos donde el tema es claro y tiene vocabulario específico (por ejemplo, hardware o motocicletas), el modelo identifica correctamente a sus vecinos más similares dentro de la misma clase.
  * En cambio, en textos con lenguaje más general o temáticas superpuestas, las similitudes son bajas y aparecen vecinos de otras categorías.
  * Esto indica que TF-IDF funciona bien cuando las palabras clave son distintivas, pero tiene limitaciones para captar el sentido semántico más profundo de los textos.

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# Cargar el conjunto de datos

newsgroups_train = fetch_20newsgroups(
    subset='train',
    remove=('headers', 'footers', 'quotes')
)

textos = newsgroups_train.data
etiquetas = newsgroups_train.target
nombres_clases = newsgroups_train.target_names

print("Cantidad de documentos:", len(textos))
print("Ejemplo de texto:\n")
print(textos[0][:500])
print()

# Elegir el tipo de vectorizador
vectorizador = TfidfVectorizer()


X = vectorizador.fit_transform(textos)

print("Forma de la matriz:", X.shape)
print("Cada fila representa un documento y cada columna una palabra distinta.")
print()

# Elegir 5 documentos especificos
indices_aleatorios = [200, 450, 3000, 3001, 10]

print("indices seleccionados:", indices_aleatorios)
print()


# Calcular la similitud del coseno entre documentos
for idx in indices_aleatorios:
    print("================================" )
    print(f"Documento base #{idx}")
    print("Clase:", nombres_clases[etiquetas[idx]])
    print("Texto (primeras 500 letras):")
    print(textos[idx][:500].replace('\n', ' ') + "...")
    print()

    # Calcular similitud del documento elegido con todos los demás
    similitudes = cosine_similarity(X[idx], X)[0]

    # Obtener los 5 indices con mayor similitud (excepto el mismo documento)
    indices_mas_similares = np.argsort(similitudes)[::-1][1:6]

    print("Documentos mas similares:")
    for i, idx_sim in enumerate(indices_mas_similares, start=1):
        print(f"{i}) Documento #{idx_sim:>6} | Similitud: {similitudes[idx_sim]:.3f} | "
              f"Clase: {nombres_clases[etiquetas[idx_sim]]}")
    print()





Cantidad de documentos: 11314
Ejemplo de texto:

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Forma de la matriz: (11314, 101631)
Cada fila representa un documento y cada columna una palabra distinta.

indices seleccionados: [200, 450, 3000, 3001, 10]

Documento base #200
Clase: talk.politics.guns
Texto (primeras 500 letras):
 I first read and consulted rec.guns in the summer of 1991.  I just purchased my first firearm in early March of this year....

Documentos mas similares:
1) Documento #  6538 | Similitud: 0.312 | Clase: talk.politics.guns
2) Documento #  48

### Punto 2

#### Observaciones:
  * El modelo de clasificación por prototipos predice cada documento del conjunto de test según la mayor similitud de coseno con los documentos de entrenamiento.
  * A diferencia de otros modelos, no aprende patrones, sino que asigna la clase del texto más parecido.
  * El proceso resulta computacionalmente costoso, ya que compara cada documento de test con todos los de train.
  * Los resultados son coherentes en categorías bien definidas, aunque el rendimiento general es bajo, reflejando las limitaciones de este método tan directo.

In [ ]:

import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score, classification_report

# Cargar datos de entrenamiento y test

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test  = fetch_20newsgroups(subset='test',  remove=('headers', 'footers', 'quotes'))

X_train_text = newsgroups_train.data
y_train = newsgroups_train.target

X_test_text  = newsgroups_test.data
y_test = newsgroups_test.target

nombres_clases = newsgroups_train.target_names


#  Vectorizar con TF-IDF
vectorizador = TfidfVectorizer()

X_train = vectorizador.fit_transform(X_train_text)
X_test  = vectorizador.transform(X_test_text)


# Clasificador por prototipos . Para cada test, buscar el train más similar
y_pred = []

print("Calculando similitudes entre TODOS los documentos de test y de train...")
sim = cosine_similarity(X_test, X_train)

# Para cada documento de test, buscamos el índice del train más parecido
idx_max = np.argmax(sim, axis=1)

# Obtenemos la clase correspondiente del train
y_pred = y_train[idx_max]


#  Evaluar el rendimiento
print("=== Resultados Clasificador por Prototipos ===")
print("F1-macro:", f1_score(y_test, y_pred, average='macro'))

print(classification_report(y_test, y_pred, target_names=nombres_clases, digits=3))


Calculando similitudes entre TODOS los documentos de test y de train...

=== Resultados Clasificador por Prototipos ===
F1-macro: 0.5049911553681621

                          precision    recall  f1-score   support

             alt.atheism      0.366     0.508     0.425       319
           comp.graphics      0.543     0.483     0.512       389
 comp.os.ms-windows.misc      0.506     0.457     0.480       394
comp.sys.ibm.pc.hardware      0.515     0.520     0.518       392
   comp.sys.mac.hardware      0.535     0.499     0.516       385
          comp.windows.x      0.701     0.592     0.642       395
            misc.forsale      0.629     0.462     0.533       390
               rec.autos      0.406     0.576     0.476       396
         rec.motorcycles      0.635     0.515     0.569       398
      rec.sport.baseball      0.645     0.537     0.586       397
        rec.sport.hockey      0.748     0.722     0.735       399
               sci.crypt      0.552     0.588     0.570  

### Punto 3

#### Observaciones:
  * El modelo Naïve Bayes logra un desempeño alto comparado con el clasificador por prototipos.
  * Se entrenaron variantes Multinomial y Complement ajustando parámetros del vectorizador y del modelo para maximizar el F1-macro.
  * Ambos alcanzan valores cercanos a 0.7 - 0.8, siendo ComplementNB ligeramente superior.
  * TF-IDF combinado con Naïve Bayes demuestra buena capacidad para separar categorías basadas en el vocabulario distintivo de cada tema.

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


# Cargar los datos
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test  = fetch_20newsgroups(subset='test',  remove=('headers', 'footers', 'quotes'))

X_train_text = newsgroups_train.data
y_train = newsgroups_train.target

X_test_text  = newsgroups_test.data
y_test = newsgroups_test.target

nombres_clases = newsgroups_train.target_names


#  Definir el pipeline (vectorizador + clasificador)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])


# Definir la grilla de hiperparámetros a probar
param_grid = {
    'tfidf__min_df': [1, 2],
    'tfidf__max_df': [0.9],
    'tfidf__sublinear_tf': [True, False],
    'tfidf__norm': ['l2', None],
    'clf__alpha': [0.5, 0.2, 0.1]
}


# Buscar la mejor combinación
grid = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=2,
    n_jobs=-1,
    verbose=1
)

print("Entrenando modelo MultinomialNB...")
grid.fit(X_train_text, y_train)

print("\nMejor combinación de parámetros:")
print(grid.best_params_)

print("Mejor F1-macro (validación cruzada):", grid.best_score_)


# Evaluar en el conjunto de test
y_pred = grid.best_estimator_.predict(X_test_text)
print("\n=== Resultados MultinomialNB en Test ===")
print("F1-macro:", f1_score(y_test, y_pred, average='macro'))
print(classification_report(y_test, y_pred, target_names=nombres_clases, digits=3))




# Ahora con ComplementNB
pipeline_cnb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', ComplementNB())
])

param_grid_cnb = {
    'tfidf__min_df': [1, 2],
    'tfidf__max_df': [0.9],
    'tfidf__sublinear_tf': [True, False],
    'tfidf__norm': ['l2', None],
    'clf__alpha': [0.5, 0.2, 0.1]
}

grid_cnb = GridSearchCV(
    pipeline_cnb,
    param_grid=param_grid_cnb,
    scoring='f1_macro',
    cv=2,
    n_jobs=-1,
    verbose=1
)

print("\nEntrenando modelo ComplementNB...")
grid_cnb.fit(X_train_text, y_train)

print("\nMejor combinación de parámetros (ComplementNB):")
print(grid_cnb.best_params_)
print("Mejor F1-macro (validación cruzada):", grid_cnb.best_score_)

y_pred_cnb = grid_cnb.best_estimator_.predict(X_test_text)
print("\n=== Resultados ComplementNB en Test ===")
print("F1-macro:", f1_score(y_test, y_pred_cnb, average='macro'))
print(classification_report(y_test, y_pred_cnb, target_names=nombres_clases, digits=3))


Entrenando modelo MultinomialNB...
Fitting 2 folds for each of 24 candidates, totalling 48 fits

Mejor combinación de parámetros:
{'clf__alpha': 0.1, 'tfidf__max_df': 0.9, 'tfidf__min_df': 2, 'tfidf__norm': 'l2', 'tfidf__sublinear_tf': False}
Mejor F1-macro (validación cruzada): 0.7065375081505362

=== Resultados MultinomialNB en Test ===
F1-macro: 0.6726552851196274
                          precision    recall  f1-score   support

             alt.atheism      0.650     0.367     0.469       319
           comp.graphics      0.645     0.715     0.678       389
 comp.os.ms-windows.misc      0.678     0.551     0.608       394
comp.sys.ibm.pc.hardware      0.614     0.722     0.664       392
   comp.sys.mac.hardware      0.747     0.696     0.720       385
          comp.windows.x      0.826     0.747     0.785       395
            misc.forsale      0.841     0.744     0.789       390
               rec.autos      0.787     0.747     0.767       396
         rec.motorcycles      0.803

### Punto 4

#### Observaciones:

  * El análisis de similitud entre palabras muestra que TF-IDF también puede representar relaciones semánticas básicas.
  * Palabras que suelen aparecer en los mismos contextos presentan alta similitud de coseno, reflejando asociaciones temáticas (por ejemplo, space-nasa, car-engine).
  * Sin embargo, el método no capta significados profundos ni sinónimos, ya que se basa solo en la coocurrencia dentro de los documentos.

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

textos = newsgroups_train.data
etiquetas = newsgroups_train.target
nombres_clases = newsgroups_train.target_names

vectorizador = TfidfVectorizer()
X = vectorizador.fit_transform(textos)
print("Matriz documento palabra:", X.shape)


# Transponer la matriz para obtener documento palabra
X_td = X.T     # ahora las filas son palabras, las columnas documentos
print("Matriz documento palabra:", X_td.shape)

# Crear diccionarios para acceder por nombre
vocabulario = vectorizador.vocabulary_             # palabra -> índice
idx_to_word = {v: k for k, v in vocabulario.items()}  # índice -> palabra


# Elegir palabras manualmente
palabras = ["space", "god", "car", "windows", "hockey"]

# Función para buscar palabras similares
def palabras_mas_similares(palabra, X_td, vocabulario, idx_to_word, k=5):

    idx = vocabulario[palabra]

    # Similaridad de coseno entre esta palabra y todas las demás
    similitudes = cosine_similarity(X_td[idx], X_td)[0]

    # Ordenar de mayor a menor y descartar la misma palabra
    indices_ordenados = np.argsort(similitudes)[::-1]
    indices_ordenados = [i for i in indices_ordenados if i != idx][:k]

    resultados = [(idx_to_word[i], similitudes[i]) for i in indices_ordenados]
    return resultados

# Mostrar los resultados
for palabra in palabras:
    print("===================")
    print(f"Palabra base: {palabra}")
    similares = palabras_mas_similares(palabra, X_td, vocabulario, idx_to_word, k=5)
    for sim_word, score in similares:
        print(f"  {sim_word:<20}  similitud: {score:.3f}")
    print()


Matriz documento palabra: (11314, 101631)
Matriz documento palabra: (101631, 11314)
Palabra base: space
  nasa                  similitud: 0.330
  seds                  similitud: 0.297
  shuttle               similitud: 0.293
  enfant                similitud: 0.280
  seti                  similitud: 0.246

Palabra base: god
  jesus                 similitud: 0.269
  bible                 similitud: 0.262
  that                  similitud: 0.256
  existence             similitud: 0.255
  christ                similitud: 0.251

Palabra base: car
  cars                  similitud: 0.180
  criterium             similitud: 0.177
  civic                 similitud: 0.175
  owner                 similitud: 0.169
  dealer                similitud: 0.168

Palabra base: windows
  dos                   similitud: 0.304
  ms                    similitud: 0.232
  microsoft             similitud: 0.222
  nt                    similitud: 0.214
  for                   similitud: 0.193

Palabra base: 